# DAT 203 - Lab Assignment #4

- Author: Colin Bowers
- Date: Jun 3, 2023

## Instructions
1. Upload Titanic dataset 
1. Define Survived column as TARGET variable 
1. Select features that can be predictive of the survival status
1. Drop features that you think are not predictive and explain why they are being dropped 
1. Transform selected categorical features with Dummy values 
1. Import logistic regression function, train and test function from sklearn library 
1. Apply logistic regression on the split train/test dataset
1. Compute your model’s accuracy using accuracy_score

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn as sk 

**1 - Upload Titanic dataset**

In [2]:
df = pd.read_csv("data/Titanic_original.csv")
df.describe(include="all").round(2)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
count,891.00,891.00,891.00,891,891,714.00,891.00,891.00,891,891.00,204,889
unique,NaN,NaN,NaN,891,2,NaN,NaN,NaN,681,NaN,147,3
top,NaN,NaN,NaN,"Braund, Mr. Owen Harris",male,NaN,NaN,NaN,347082,NaN,B96 B98,S
freq,NaN,NaN,NaN,1,577,NaN,NaN,NaN,7,NaN,4,644
mean,446.00,0.38,2.31,NaN,NaN,29.70,0.52,0.38,NaN,32.20,NaN,NaN
std,257.35,0.49,0.84,NaN,NaN,14.53,1.10,0.81,NaN,49.69,NaN,NaN
min,1.00,0.00,1.00,NaN,NaN,0.42,0.00,0.00,NaN,0.00,NaN,NaN
25%,223.50,0.00,2.00,NaN,NaN,20.12,0.00,0.00,NaN,7.91,NaN,NaN
50%,446.00,0.00,3.00,NaN,NaN,28.00,0.00,0.00,NaN,14.45,NaN,NaN
75%,668.50,1.00,3.00,NaN,NaN,38.00,1.00,0.00,NaN,31.00,NaN,NaN


**2 - Define Survived column as TARGET variable**

In [3]:
# X contains independent values, y contains dependent value
target = 'Survived'

**3 - Select features that can be predictive of the survival status**

In [4]:
predictors = ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'Sex', 'Embarked']
#predictors = ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare']

**4 - Drop features that you think are not predictive and explain why they are being dropped**

In [5]:
# To reduce column count, let's use PassengerId as the index
df.set_index("PassengerId", inplace=True)

# Logistic Regression requires numeric variables so we should drop these
df.drop('Name', axis=1, inplace=True)
df.drop('Ticket', axis=1, inplace=True)
df.drop('Cabin', axis=1, inplace=True)

# OTHER POSSIBLE FEATURES 
# Deck level could be extracted from Cabin which could be a significant predictor
# Marital status could be extracted from Name (i.e. Ms. vs Mrs.)
# Other titles and social status could be extracted from Name (e.g. Dr.)


In [6]:
# Fill missing values for Age using the mode
mode = df["Age"].mode()[0]
df.fillna({'Age': mode}, inplace=True)

In [7]:
df.describe().round(1)

,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.0,891.0,891.0,891.0,891.0,891.0
mean,0.4,2.3,28.6,0.5,0.4,32.2
std,0.5,0.8,13.2,1.1,0.8,49.7
min,0.0,1.0,0.4,0.0,0.0,0.0
25%,0.0,2.0,22.0,0.0,0.0,7.9
50%,0.0,3.0,24.0,0.0,0.0,14.5
75%,1.0,3.0,35.0,1.0,0.0,31.0
max,1.0,3.0,80.0,8.0,6.0,512.3


**5 - Transform selected categorical features with Dummy values**

In [8]:
dummies = ['Embarked', 'Sex']
for d in dummies:
    df[d] = df[d].astype('category')

dummy_data = pd.get_dummies(df[dummies])
df2 = pd.concat([df, dummy_data], axis=1)
df2.drop(dummies, axis=1, inplace=True)

df2.head()

,Survived,Pclass,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S,Sex_female,Sex_male
PassengerId,,,,,,,,,,,
1,0,3,22.0,1,0,7.2500,0,0,1,0,1
2,1,1,38.0,1,0,71.2833,1,0,0,1,0
3,1,3,26.0,0,0,7.9250,0,0,1,1,0
4,1,1,35.0,1,0,53.1000,0,0,1,1,0
5,0,3,35.0,0,0,8.0500,0,0,1,0,1


In [9]:
predictors.remove("Embarked")
predictors.remove("Sex")
predictors += ["Embarked_C", "Embarked_Q", "Embarked_S", "Sex_female", "Sex_male" ]
predictors

['Pclass',
 'Age',
 'SibSp',
 'Parch',
 'Fare',
 'Embarked_C',
 'Embarked_Q',
 'Embarked_S',
 'Sex_female',
 'Sex_male']

**6 - Import logistic regression function, train and test function from sklearn library**

In [10]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [11]:
train, test = train_test_split(df2, test_size=0.2)
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 712 entries, 348 to 445
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Survived    712 non-null    int64  
 1   Pclass      712 non-null    int64  
 2   Age         712 non-null    float64
 3   SibSp       712 non-null    int64  
 4   Parch       712 non-null    int64  
 5   Fare        712 non-null    float64
 6   Embarked_C  712 non-null    uint8  
 7   Embarked_Q  712 non-null    uint8  
 8   Embarked_S  712 non-null    uint8  
 9   Sex_female  712 non-null    uint8  
 10  Sex_male    712 non-null    uint8  
dtypes: float64(2), int64(4), uint8(5)
memory usage: 42.4 KB


**7 - Apply logistic regression on the split train/test dataset**

In [12]:
X_train = train[predictors]
y_train = train[target]

# Initially, this was generating the error: "ConvergenceWarning: lbfgs failed to converge"
# Therefore, increased max_iter
model = LogisticRegression(solver='lbfgs', max_iter=500)
model.fit(X_train, y_train)

model.coef_

array([[-1.04811343, -0.0459236 , -0.38480766, -0.10019839,  0.00409089,
         0.16084438, -0.00799877, -0.14987766,  1.38567564, -1.38270769]])

In [13]:
X_test = test[predictors]
y_test = test[target]

y_pred = model.predict(X_test)
y_pred


array([0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0,
       0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0,
       1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0,
       1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1,
       1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0,
       0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 1])

**8 - Compute your model’s accuracy using accuracy_score**

In [14]:
from sklearn.metrics import accuracy_score, confusion_matrix

In [15]:
confusion_matrix(y_pred, y_test)

array([[88, 25],
       [17, 49]])

Total wrong predictions: 28 + 17 = 45

In [16]:
acc = accuracy_score(y_pred, y_test)
print(f"Accuracy: {acc*100:.2f}%")

Accuracy: 76.54%
